This notebook has been imported from Kaggle. Please note that the dataset used in this notebook is not included here. However, you can find the dataset [here](https://www.kaggle.com/datasets/joebeachcapital/realwaste).
